## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,80.53,83,20,5.75,few clouds
1,1,Khatanga,RU,71.9667,102.5000,-1.86,97,99,10.25,overcast clouds
2,2,Faya,SA,18.3851,42.4509,61.32,74,6,3.47,clear sky
3,3,Lashio,MM,22.9333,97.7500,56.10,96,0,1.39,clear sky
4,4,Burnie,AU,-41.0667,145.9167,51.03,75,88,8.08,overcast clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?40
What is the maximum temperature you would like for your trip?90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)& \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,80.53,83,20,5.75,few clouds
2,2,Faya,SA,18.3851,42.4509,61.32,74,6,3.47,clear sky
3,3,Lashio,MM,22.9333,97.7500,56.10,96,0,1.39,clear sky
4,4,Burnie,AU,-41.0667,145.9167,51.03,75,88,8.08,overcast clouds
5,5,Kaitangata,NZ,-46.2817,169.8464,42.91,94,100,4.25,overcast clouds
6,6,Itoman,JP,26.1247,127.6694,68.02,73,75,11.50,broken clouds
7,7,Codrington,AU,-38.2667,141.9667,49.84,88,13,10.27,few clouds
8,8,Hithadhoo,MV,-0.6000,73.0833,81.12,71,76,13.58,broken clouds
9,9,Perpignan,FR,42.7500,2.7500,46.04,53,0,14.97,clear sky
11,11,Comodoro Rivadavia,AR,-45.8667,-67.5000,60.66,41,20,14.97,few clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                477
City                   477
Country                473
Lat                    477
Lng                    477
Max Temp               477
Humidity               477
Cloudiness             477
Wind Speed             477
Current Description    477
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,80.53,83,20,5.75,few clouds
2,2,Faya,SA,18.3851,42.4509,61.32,74,6,3.47,clear sky
3,3,Lashio,MM,22.9333,97.7500,56.10,96,0,1.39,clear sky
4,4,Burnie,AU,-41.0667,145.9167,51.03,75,88,8.08,overcast clouds
5,5,Kaitangata,NZ,-46.2817,169.8464,42.91,94,100,4.25,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
681,681,Salimpur,IN,28.1280,76.2766,65.43,42,0,3.58,clear sky
684,684,Yeppoon,AU,-23.1333,150.7333,74.98,90,100,3.65,overcast clouds
685,685,Casino,AU,-28.8667,153.0500,67.95,94,95,2.30,overcast clouds
688,688,Gimry,RU,42.7597,46.8396,40.73,88,87,2.21,overcast clouds


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
2,Faya,SA,61.32,clear sky,18.3851,42.4509,
3,Lashio,MM,56.10,clear sky,22.9333,97.7500,
4,Burnie,AU,51.03,overcast clouds,-41.0667,145.9167,
5,Kaitangata,NZ,42.91,overcast clouds,-46.2817,169.8464,
6,Itoman,JP,68.02,broken clouds,26.1247,127.6694,
7,Codrington,AU,49.84,few clouds,-38.2667,141.9667,
8,Hithadhoo,MV,81.12,broken clouds,-0.6000,73.0833,
9,Perpignan,FR,46.04,clear sky,42.7500,2.7500,
11,Comodoro Rivadavia,AR,60.66,few clouds,-45.8667,-67.5000,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.....skipping.")
        
        

Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"]!='']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
2,Faya,SA,61.32,clear sky,18.3851,42.4509,املاك خالد الشعشوع
3,Lashio,MM,56.10,clear sky,22.9333,97.7500,Hotel CF Lashio - Burmese Only
4,Burnie,AU,51.03,overcast clouds,-41.0667,145.9167,Beachfront Voyager Motor Inn
5,Kaitangata,NZ,42.91,overcast clouds,-46.2817,169.8464,Kaitangata Motor Camp
...,...,...,...,...,...,...,...
681,Salimpur,IN,65.43,clear sky,28.1280,76.2766,जाजम Royal Family Rao shab
684,Yeppoon,AU,74.98,overcast clouds,-23.1333,150.7333,Villa Mar Colina
685,Casino,AU,67.95,overcast clouds,-28.8667,153.0500,Squatters Homestead Motel
688,Gimry,RU,40.73,overcast clouds,42.7597,46.8396,Yakht Klub Kan'on


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))